# Run model notebook

In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import skimage.io

# Root directory of the project
ROOT_DIR = os.path.abspath("../../Mask_RCNN")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
sys.path.append(os.path.abspath(".."))
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
BADMINTON_MODEL_PATH = os.path.abspath("../detection/weights.h5")
BADMINTON_DATASET_DIR = os.path.abspath("../dataset")


from detection.detection_config import DetectionConfig
from dataset.badminton_dataset import BadmintonDataset
print("Done.")

## Configurations

In [ ]:
config = DetectionConfig()
print("Done.")

## Create Model and Load Trained Weights

In [ ]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(BADMINTON_MODEL_PATH, by_name=True)

print("Done.")

---
## Datasets

### Random train

In [ ]:
IMAGE_DIR = os.path.join(BADMINTON_DATASET_DIR, "train/")
dataset_train = BadmintonDataset()
dataset_train.load_badminton(BADMINTON_DATASET_DIR, "train")
dataset_train.prepare()

print("Image Count: {}".format(len(dataset_train.image_ids)))
print("Loading image...")
file_names = next(os.walk(IMAGE_DIR))[2]
image = skimage.io.imread(os.path.join(IMAGE_DIR, random.choice(file_names)))

print("Running detecion...")
results = model.detect([image], verbose=1)

print("Visualizing results...")
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_train.class_names, r['scores'])

print("Done.")

### Random val

In [ ]:
IMAGE_DIR = os.path.join(BADMINTON_DATASET_DIR, "val/")
dataset = BadmintonDataset()
dataset.load_badminton(BADMINTON_DATASET_DIR, "val")
dataset.prepare()
print("Image Count: {}".format(len(dataset.image_ids)))

print("Loading image...")
file_names = next(os.walk(IMAGE_DIR))[2]
image = skimage.io.imread(os.path.join(IMAGE_DIR, random.choice(file_names)))

print("Running detecion...")
results = model.detect([image], verbose=1)

print("Visualizing results...")
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            dataset.class_names, r['scores'])

print("Done.")

### Test images

In [ ]:
IMAGE_DIR = os.path.join(BADMINTON_DATASET_DIR, "test/")
file_names = next(os.walk(IMAGE_DIR))[2]

print("Loading images...")
images = []
for file_name in file_names:
    if file_name.endswith('.jpg') or file_name.endswith('.png'):
        image = skimage.io.imread(os.path.join(IMAGE_DIR, file_name))
        images.append(image)
        print(file_name)
print("Loaded {} images.".format(len(images)))

for index in range(len(images)):
    print("Image {}/{}".format(index+1, len(images)))
    image = images[index]
    results = model.detect([image], verbose=1)
    r = results[0]
    visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            dataset.class_names, r['scores'])

print("Done.")